In [ ]:
import pandas as pd
import warnings
import io
import ipywidgets as widgets
from IPython.display import display, HTML
import os, base64

warnings.filterwarnings("ignore")

# --- Only Quote upload widget ---
upload_quote = widgets.FileUpload(accept='.xls,.xlsx', multiple=False, description="Upload Quote File")

# --- Output filename widget ---
output_name = widgets.Text(
    value="Final_Report.xlsx",
    description="Output File:",
    style={'description_width': 'initial'},
    placeholder="Enter output file name (e.g., Report.xlsx)"
)

# --- Run button & output area ---
run_button = widgets.Button(description="Run Reports", button_style="success")
output = widgets.Output()

display(upload_quote, output_name, run_button, output)

# --- File helper for Quote ---
def file_to_df(upload_widget, skiprows=0):
    if not upload_widget.value:
        return None
    if isinstance(upload_widget.value, dict):  # ipywidgets 8
        for fname, file_info in upload_widget.value.items():
            return pd.read_excel(io.BytesIO(file_info['content']))
    elif isinstance(upload_widget.value, (tuple, list)):  # Voilà
        file_info = upload_widget.value[0]
        return pd.read_excel(io.BytesIO(file_info['content']))
    return None

# --- Format functions ---
def formats(d):
    return pd.DataFrame({
        'MAJOR': d['Part Number'],
        'MINOR': d['Unnamed: 1'],
        'Product': d['Product'],
        'Product Description': d['Product Description'],
        'Price in USD': d['Price in USD'],
        'End of Sale Date': d['End Of Sale Date'],
        'ERP Family': d['ERP Family'],
        'Discount': d['Discount %']
    })

def formats2(d):
    return pd.DataFrame({
        'MAJOR': d['Part Number'],
        'MINOR': d['Unnamed: 1'],
        'Product': d['Product'],
        'Product Description': d['Product Description'],
        'Price in USD': d['Global List Price '],
        'End of Sale Date': d['End Of Sale Date'],
        'ERP Family': d['ERP Family'],
        'Discount': d['Discount %']
    })

# --- Download link generator ---
def create_download_link(filepath, description="📥 Click here to download the report"):
    with open(filepath, "rb") as f:
        data = f.read()
    b64 = base64.b64encode(data).decode()
    href = f'<a download="{os.path.basename(filepath)}" href="data:application/octet-stream;base64,{b64}">{description}</a>'
    return HTML(href)

# --- Extract Part Number + Discount % from Quote ---
def extract_quote_to_rnsd(quote_df):
    # Locate the row where "Special Discount Items" starts
    idx = quote_df[quote_df.iloc[:,0].astype(str).str.contains("Special Discount Items", na=False)].index
    if len(idx) == 0:
        raise ValueError("❌ Could not find 'Special Discount Items' in Quote file.")
    start_idx = idx[0] + 1  # data is after this row

    # Slice rows below this section
    sliced = quote_df.iloc[start_idx:].copy()

    # Keep only Product Family + Discount% columns
    # (adjust column names if different in your Quote file)
    if "Product Family" not in sliced.columns or "Discount%" not in sliced.columns:
        raise ValueError("❌ Could not find 'Product Family' or 'Discount%' columns in Quote file.")

    rnsd_df = sliced[["Product Family", "Discount%"]].dropna()
    rnsd_df = rnsd_df.rename(columns={"Product Family": "Part Number", "Discount%": "Discount %"})
    return rnsd_df

# --- Main pipeline ---
def run_reports(b):
    with output:
        output.clear_output()
        try:
            print("▶️ Starting report generation...")

            # Step 1: Read Quote upload
            quote_df = file_to_df(upload_quote)
            if quote_df is None:
                print("⚠️ Please upload the Quote file before running.")
                return

            print("✅ Quote uploaded. Extracting Part Number + Discount %...")

            # Step 2: Extract custom RNSD-like file
            rnsd = extract_quote_to_rnsd(quote_df)
            rnsd['Part Number'] = rnsd['Part Number'].astype(str).str.upper()

            print("✅ Extracted RNSD from Quote file:")
            display(rnsd.head())

            # Step 3: Load prod & xaas from repo
            prod = pd.read_excel("prod_glus.xlsx", skiprows=1)
            xaas = pd.read_excel("xaas_glus.xlsx", skiprows=2)

            # Step 4: Merge logic
            prod = prod.rename(columns={'Unnamed: 0': 'Part Number'})
            prod['Part Number'] = prod['Part Number'].str.upper()
            left1 = pd.merge(prod, rnsd, on='Part Number', how='inner')

            xaas = xaas.rename(columns={'Unnamed: 0': 'Part Number'})
            xaas['Part Number'] = xaas['Part Number'].str.upper()
            left3 = pd.merge(xaas, rnsd, on='Part Number', how='inner')

            left1 = formats(left1)
            left3 = formats2(left3)

            # Step 5: Combine
            full = pd.merge(left1, left3, how='outer')
            full.drop(columns=['Price in USD', 'End of Sale Date', 'MINOR'], inplace=True)
            full = full[full['ERP Family'].str.upper() != 'SERVICE']

            print("✅ Merge complete. Saving to Excel...")

            # Step 6: Export
            final_name = output_name.value if output_name.value.endswith(".xlsx") else output_name.value + ".xlsx"
            writer = pd.ExcelWriter(final_name, engine='xlsxwriter')
            full.to_excel(writer, index=False, sheet_name="Item Mapping")
            workbook = writer.book
            worksheet = writer.sheets["Item Mapping"]

            (max_row, max_col) = full.shape
            column_settings = [{'header': col} for col in full.columns]
            worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings, 'style': None})
            worksheet.set_column(0, max_col, 12)
            writer.close()

            print(f"✅ Report generated successfully: {final_name}")
            display(full.head())

            # Step 7: Download link
            display(create_download_link(final_name))

        except Exception as e:
            import traceback
            print("❌ An error occurred while running the report:")
            print(str(e))
            print(traceback.format_exc())

# Connect button
run_button.on_click(run_reports)